In [1]:
import os

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

In [2]:
directory_in = './file/3.part'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'1.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：450



,No,Part Number,Vehicle,Picture,Url,Json_Type,Json_Info,Json_Specification
0,1,APK100,Cadillac Escalade (03)\nChevrolet Avalanche 15...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
1,2,APK101,Cadillac Escalade (05-03)\nCadillac Escalade E...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
2,3,APK102,Chevrolet B7 (02-97)\nChevrolet Blazer (94)\nC...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
3,4,APK103,Ford F-250 (96)\nFord F-250 HD (97)\nFord F-25...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
4,5,APS101,Ford F-250 (96)\nFord F-350 (97-96)\nFord F-59...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
...,...,...,...,...,...,...,...,...
445,446,APS717,Lexus LX600 (24-22),,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STI"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
446,447,G92003,Cadillac Escalade (03)\nChevrolet Avalanche 15...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""TEC"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
447,448,G92004,Cadillac Escalade (05-03)\nCadillac Escalade E...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""TEC"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."
448,449,G92005,Chevrolet B7 (02-97)\nChevrolet Blazer (94)\nC...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""TEC"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                                    1
 Part Number                                                      APK100
 Vehicle               Cadillac Escalade (03)\nChevrolet Avalanche 15...
 Picture                                                                
 Url                   https://www.standardbrand.com/en/ecatalog?part...
 Json_Type                           {"0": "Sensor - Accelerator Pedal"}
 Json_Info             {"Brand": "STD", "Partdesc_En": "Accelerator P...
 Json_Specification    {"0": {"Warranty": "3 years/36,000 mile"}, "1"...
 Name: 0, dtype: object)

In [5]:
import time

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?partNum={crawler_series['Part Number']}&brand={json.loads(crawler_series['Json_Info'])['Brand']}&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960''', crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp.json()

1


[{'image_Name': '3EXzJVEsi510QFHvWujpJw%3D%3D',
  'image_SM_Url': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1OUojjz7Fe0uu7MlzNMTtPOA3FM8wSUCfEFLjhl+yY51Wji2BTIoaJjOd4unmb3P8rLuwF0NJIf55URlMOW+XVqOzvag35n6IYODDVJXQ/5aAUVD7HPvINfMHmFkfTlA8D+msAMmvX2CZuqvooaa7DCB6ftZHycUfh/RSgS97TYr+m7fx4OW9L38hy3Ug/3UipkBNHJMFz+NdB7H8SFNzJiQJSOi3Hj/',
  'image_MD_Url': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1OUojjz7Fe0uu7MlzNMTtPOA3FM8wSUCqYrcgaXQHScWSaSu79Q7PWmsSG3zyFfwYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0HoI9kWyXUlSG47DcAIwETg06j7hX67oM0bbvjBiwbVe1wM0rktTvG9cAwB44CkapgH',
  'image_BG_Url': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1OUojjz7Fe0uu7MlzNMTtPOA3FM8wSUCTuE74rL23eTm8CXSh6KpKE+ijFXaAgeWnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCdQCfVj8UUYFH/vcGamvfKrZu7Cn8K35U+X7Yt0dsg+ooOsi5MlAbZptp5EMIddhrA=='},
 {'image_Name': '3EXzJVEsi505CCqo6fpf%2Fw%3D%3D',


In [6]:
dict_src = {}
for i, dict_ in enumerate(resp.json()):
    dict_src[str(i)] = dict_['image_BG_Url'].strip()

dict_src

{'0': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1OUojjz7Fe0uu7MlzNMTtPOA3FM8wSUCTuE74rL23eTm8CXSh6KpKE+ijFXaAgeWnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCdQCfVj8UUYFH/vcGamvfKrZu7Cn8K35U+X7Yt0dsg+ooOsi5MlAbZptp5EMIddhrA==',
 '1': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1OUojjz7Fe0uu7MlzNMTtPOA3FM8wSUCTuE74rL23eTm8CXSh6KpKHYzHaHL6R2Yrm+ZHodTWPtGHGwNK6wuSZiH9+8ykTJFaACzcM2Fwb3lQjHfxmkEu/M2DCKxWi0p8RRteh9+Smi4/XfVIJ38wXfbMVRlDB7P783s11OnTH0m+illfMglRS2o5CJbOjqSel2Wedg4OWSUt85nNN2TLw==',
 '2': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1OUojjz7Fe0uu7MlzNMTtPOA3FM8wSUCTuE74rL23eTm8CXSh6KpKCVh+luC94SgnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCdQCfVj8UUYFH/vcGamvfKrZu7Cn8K35U+X7Yt0dsg+ooOsi5MlAbZptp5EMIddhrA=='}

In [7]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Part Number': crawler_series['Part Number'],
                          'Vehicle': crawler_series['Vehicle'],
                          'Picture': crawler_series['Picture'],
                          'Url': crawler_series['Url'],
                          'Json_Src': json.dumps(dict_src),
                          'Json_Type': crawler_series['Json_Type'],
                          'Json_Info': crawler_series['Json_Info'],
                          'Json_Specification': crawler_series['Json_Specification']})

list_dict_correct

[{'No': 1,
  'Part Number': 'APK100',
  'Vehicle': 'Cadillac Escalade (03)\nChevrolet Avalanche 1500 (03)\nChevrolet Avalanche 2500 (03)\nChevrolet Silverado 1500 (03)\nChevrolet Silverado 1500 HD (03)\nChevrolet Silverado 2500 (03)\nChevrolet Silverado 2500 HD (05-03)\nChevrolet Silverado 3500 (05-03)\nChevrolet Suburban 1500 (03)\nChevrolet Suburban 2500 (03)\nChevrolet Tahoe (03)\nGMC Sierra 1500 (03-01)\nGMC Sierra 1500 HD (03-01)\nGMC Sierra 2500 (03-00)\nGMC Sierra 2500 HD (05-01)\nGMC Sierra 3500 (05-01)\nGMC Yukon (03-00)\nGMC Yukon XL 1500 (03-00)\nGMC Yukon XL 2500 (03-00)\nHummer H2 (03)',
  'Picture': '',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=APK100&type=p&search=s',
  'Json_Src': '{"0": "https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1OUojjz7Fe0uu7MlzNMTtPOA3FM8wSUCTuE74rL23eTm8CXSh6KpKE+ijFXaAgeWnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCdQCfVj8UUYFH/vcGamvfKrZu7Cn8K35U+X7Yt0dsg+ooOsi5MlAbZptp5EMIddhrA

In [8]:
crawler_status = True

crawler_status

True

In [9]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                                    1
 Part Number                                                      APK100
 Vehicle               Cadillac Escalade (03)\nChevrolet Avalanche 15...
 Picture                                                                
 Url                   https://www.standardbrand.com/en/ecatalog?part...
 Json_Type                           {"0": "Sensor - Accelerator Pedal"}
 Json_Info             {"Brand": "STD", "Partdesc_En": "Accelerator P...
 Json_Specification    {"0": {"Warranty": "3 years/36,000 mile"}, "1"...
 Request_Url           https://ecatalog.smpcorp.com/V2/STD/api/image/...
 Check_Url             https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object]

In [10]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_src.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Part Number,Vehicle,Picture,Url,Json_Src,Json_Type,Json_Info,Json_Specification
0,1,APK100,Cadillac Escalade (03)\nChevrolet Avalanche 15...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""https://ecatalog.smpcorp.com/ihgs/image...","{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [11]:
print('Output Finished !')
print()

Output Finished !



In [12]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_src_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Part Number,Vehicle,Picture,Url,Json_Type,Json_Info,Json_Specification,Request_Url,Check_Url
0,1,APK100,Cadillac Escalade (03)\nChevrolet Avalanche 15...,,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Sensor - Accelerator Pedal""}","{""Brand"": ""STD"", ""Partdesc_En"": ""Accelerator P...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""...",https://ecatalog.smpcorp.com/V2/STD/api/image/...,https://www.standardbrand.com/en/ecatalog?part...


In [13]:
print('Done ~')

Done ~
